In [ ]:
# Reload the notebook if an external file is updated
%load_ext autoreload
%autoreload 2

import sys

from pathlib import Path

path = str(Path.cwd().parent)
sys.path.append(path)

In [ ]:
import numpy as np

from constant import CWD, SETTINGS
from copy import deepcopy
from datatype.dataset import Dataset
from datatype.settings import resolve, Settings
from datatype.spectrogram import Linear, Mel, Spectrogram
from io import BytesIO
from itertools import permutations
from PIL import Image, ImageDraw, ImageEnhance, ImageFilter, ImageFont, ImageOps

In [ ]:
def create_grid(collection, method):
    row = len(collection)
    column = 1
    
    width, height = np.vectorize(lambda x: x.size)(collection)
    width = np.amax(width)
    height = np.amax(height)

    border = 75
    offset = 10
    
    grid = Image.new(
        'L', 
        color='white',
        size=(
            (column * width) + offset, 
            (row * height) + (offset * 2)
        )
    )
    
    for index, image in enumerate(collection, 0):
        grid.paste(
            image, 
            box=(
                (index % column * width) + offset, 
                (index // column * height) + (offset * 2)
            )
        )

    grid = ImageOps.expand(
        grid, 
        border=border, 
        fill=(255)
    )
    
    draw = ImageDraw.Draw(grid)
    font = ImageFont.truetype("FONTS/arial.ttf", 36)
    
    w, h = grid.size

    draw.text(
        (w // 2, 10),
        method,
        (0),
        font=font
    )
    
    return grid

In [ ]:
def create_image(spectrogram):
    # spectrogram[
    #     np.where(spectrogram > [100])
    # ] = [255]
        
    buffer = BytesIO()
    image = Image.fromarray(~spectrogram)
    
    # median = ImageFilter.MaxFilter(size=3)
    # image = image.filter(median)

    return image

In [ ]:
def create_spectrogram(signal, settings):
    spectrogram = Spectrogram()
    strategy = Mel(signal, settings)
    spectrogram.strategy = strategy

    return spectrogram.generate()

In [ ]:
dataset = Dataset('signal')
dataframe = dataset.load()

In [ ]:
path = SETTINGS.joinpath('dereverberate.json')
dereverberate = Settings.from_file(path)

In [ ]:
path = SETTINGS.joinpath('spectrogram.json')
settings = Settings.from_file(path)

In [ ]:
minimum = 10
maximum = minimum + 10

subset = dataframe.iloc[minimum:maximum]
# subset = dataframe.sample(100)

In [ ]:
segmentation = subset.segmentation.to_numpy()

In [ ]:
segmentation = np.frompyfunc(
    lambda x: resolve(x),
    1,
    1
)(segmentation)

In [ ]:
callback = {
    'dereverberate': np.frompyfunc(
        lambda x: x.dereverberate(dereverberate),
        1,
        0
    ),
    
    'filter': np.frompyfunc(
        lambda x, y: x.filter(
            y.butter_lowcut,
            y.butter_highcut
        ),
        2,
        0
    ),
    
    'normalize': np.frompyfunc(
        lambda x: x.normalize(),
        1,
        0
    ),
    
    # 'reduce': np.frompyfunc(
    #     lambda x: x.reduce(),
    #     1,
    #     0
    # )
}

In [ ]:
keys = [*callback]

methods = []

for i in range(0, len(keys) + 1):
    for permutation in permutations(keys, i):
        if permutation:
            methods.append(permutation)

In [ ]:
for index, method in enumerate(methods, 0):
    index = str(index).zfill(2)
    base = '_'.join(method)
    filename = f"{index}_{base}.png"
    
    signal = deepcopy(
        subset.signal.to_numpy()
    )

    for function in method:
        if function == 'filter':
            callback[function](signal, segmentation)
        else:
            callback[function](signal)
            
    spectrogram = np.frompyfunc(
        lambda x, y: create_spectrogram(x, y),
        2,
        1
    )(signal, settings)

    image = np.frompyfunc(
        lambda x: create_image(x),
        1,
        1
    )(spectrogram)

    grid = create_grid(image, base)
    grid.save(filename, format='png')